# **Tutorial 1: Setting Up and Training SIDISH**
This tutorial explains how to initialize and train the SIDISH framework. SIDISH combines a Variational Autoencoder (VAE) and Deep Cox regression to uncover high-risk cell populations and predict clinical outcomes.
### In this tutorial, you will:
- Load the prepared data from Tutorial 0.  
- Initialize SIDISH with appropriate model architecture, learning rates, and optimizer settings. 
### Outcome:
By the end of this tutorial, you will have a fully trained SIDISH model capable of identifying high-risk cell subpopulations and linking them to clinical outcomes.


## **Step 1: Environment Setup**


### **1.1 Install Required Libraries**
To ensure compatibility, SIDISH requires Python 3.9 or higher. For best results, we recommend creating a virtual environment to manage dependencies:

Create a conda environment:
```bash
conda create --name sidish_env
```
Activate the environment:
```bash
conda activate sidish_env
```

### **1.2 Install SIDISH**
Once the environment is active, install dependencies using:

```bash
pip install -r requirements.txt
```
Then to install SIDISH, run:

```bash
pip install SIDISH==1.0.0
```

## **Step 2: Import libraries**
### **2.1 Import SIDISH**
The SIDISH framework is imported directly for use:

In [16]:
from SIDISH import SIDISH as sidish

### **2.2 Import Additional Libraries**
Additional libraries for data handling, visualization, and deep learning are required:

In [17]:
import scanpy as sc
import pandas as pd
import numpy as np
import  torch
import random
import os
import matplotlib.pyplot as plt

### **2.3 Set Seeds**
To ensure results are consistent across multiple runs, set seeds for all key libraries:

In [18]:
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(1)
ite = 0
# Set seeds for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Call the seed setting function
set_seed(seed)

## **Step 3: Reading Data sets**
We load the processed data prepared in Tutorial 0. Loading both the scRNA-seq and bulk RNA-seq data ensures that SIDISH has access to the required datasets for training.

### **3.1 Read single-cell data**
SIDISH requires initialization before training. Each phase must be set up with appropriate parameters.

In [19]:
# Read single-cell RNA-seq data
adata = sc.read_h5ad("../../DATA/processed_adata.h5ad")

### **3.2 Read bulk and survival data**

In [22]:
# Read bulk RNA-seq
bulk = pd.read_csv("../../DATA/processed_bulk.csv", index_col=0)

## **Step 4: Initializing SIDISH**
Initialise SIDISH model with the saved single-cell and merged bulk RNA-seq data.

In [24]:
sdh = sidish(adata, bulk, "cpu", seed=ite)

### **4.1 Initialise Phase 1 of SIDISH**

In [7]:
sdh.init_Phase1(225, 20, 32, [512, 128], 512, "Adam", 1.0e-4, 1e-4, 0)

### **4.2 Initialise Phase 2 of SIDISH**

In [8]:
sdh.init_Phase2(500, 128, 1e-4, 0, 0.2, 256)

## **Step 5: Start Training SIDISH**

In [ ]:
train_adata = sdh.train(5, 0.95, 30, "./LUNG/")

########################################## ITERATION 1 OUT OF 5 ##########################################
[epoch 000]  average training loss: 1018.3363
[epoch 001]  average training loss: 918.3922
[epoch 002]  average training loss: 882.5828
[epoch 003]  average training loss: 869.5693
[epoch 004]  average training loss: 864.7255
[epoch 005]  average training loss: 862.7633
[epoch 006]  average training loss: 861.7798
[epoch 007]  average training loss: 860.8265
[epoch 008]  average training loss: 859.3374
[epoch 009]  average training loss: 855.6150
[epoch 010]  average training loss: 843.3806
[epoch 011]  average training loss: 808.7727
[epoch 012]  average training loss: 768.3458
[epoch 013]  average training loss: 749.0256
[epoch 014]  average training loss: 739.9565
[epoch 015]  average training loss: 735.1591
[epoch 016]  average training loss: 732.3251
[epoch 017]  average training loss: 730.0145
[epoch 018]  average training loss: 728.7105
[epoch 019]  average training loss: 7